In [ ]:
!pip install -U diffusers transformers accelerate huggingface_hub

In [5]:
import requests
import time
from PIL import Image
import io

# --- Configuration ---
API_URL = "https://api-inference.huggingface.co/models/black-forest-labs/FLUX.1-dev"
API_TOKEN = ""  # Replace with your Hugging Face token
headers = {"Authorization": f"Bearer {API_TOKEN}"}

captions = [
    "A futuristic cityscape at sunset with flying cars",
    "A cat wearing sunglasses sitting on a beach",
    "A fantasy castle floating in the sky",
    "An astronaut riding a horse on Mars",
    "A bowl of ramen with glowing noodles in a cyberpunk setting"
]

def query_flux_api(prompt, retries=3, delay=5):
    """Query HF API with basic retry logic"""
    payload = {
        "inputs": prompt,
        "parameters": {
            "height": 512,
            "width": 512,
            "guidance_scale": 3.5,
            "num_inference_steps": 20
        }
    }

    for attempt in range(retries):
        response = requests.post(API_URL, headers=headers, json=payload)

        if response.status_code == 200:
            return response.content

        print(f"   Attempt {attempt + 1} failed (HTTP {response.status_code})")
        time.sleep(delay)

    return None

# --- Throughput Time Calculation ---
successful = 0
start_time = time.time()

for i, caption in enumerate(captions):
    print(f"Generating {i+1}/{len(captions)}: {caption}")
    image_data = query_flux_api(caption)

    if image_data:
        try:
            image = Image.open(io.BytesIO(image_data))
            image.verify()  # Confirm it's a valid image
            print(f"    Image generated. Size: {image.size}")
            successful += 1
        except:
            print("    Invalid image data received")
    else:
        print("    Failed to generate image after retries")

end_time = time.time()

# --- Final Throughput Metrics ---
total_time = end_time - start_time

if successful > 0:
    throughput = successful / total_time
    print(f"\nTotal time for {successful} successful generations: {total_time:.2f} seconds")
    print(f"Throughput: {throughput:.2f} images per second")
else:
    print("\n No successful generations. Throughput cannot be calculated.")


Generating 1/5: A futuristic cityscape at sunset with flying cars
    Image generated. Size: (512, 512)
Generating 2/5: A cat wearing sunglasses sitting on a beach
    Image generated. Size: (512, 512)
Generating 3/5: A fantasy castle floating in the sky
    Image generated. Size: (512, 512)
Generating 4/5: An astronaut riding a horse on Mars
    Image generated. Size: (512, 512)
Generating 5/5: A bowl of ramen with glowing noodles in a cyberpunk setting
    Image generated. Size: (512, 512)

Total time for 5 successful generations: 49.61 seconds
Throughput: 0.10 images per second
